In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# 给定开源数据集enron1；

## 查看下合法邮件和垃圾邮件的内容 

In [2]:
file_path = 'enron1/ham/0007.1999-12-14.farmer.ham.txt'
with open(file_path,'r') as infile:
    ham_sample = infile.read()
print(ham_sample)

Subject: mcmullen gas for 11 / 99
jackie ,
since the inlet to 3 river plant is shut in on 10 / 19 / 99 ( the last day of
flow ) :
at what meter is the mcmullen gas being diverted to ?
at what meter is hpl buying the residue gas ? ( this is the gas from teco ,
vastar , vintage , tejones , and swift )
i still see active deals at meter 3405 in path manager for teco , vastar ,
vintage , tejones , and swift
i also see gas scheduled in pops at meter 3404 and 3405 .
please advice . we need to resolve this as soon as possible so settlement
can send out payments .
thanks


In [3]:
file_path = 'enron1/spam/0058.2003-12-21.GP.spam.txt'
with open(file_path,'r') as infile:
    spam_sample = infile.read()
print(spam_sample)

Subject: stacey automated system generating 8 k per week parallelogram
people are
getting rich using this system ! now it ' s your
turn !
we ' ve
cracked the code and will show you . . . .
this is the
only system that does everything for you , so you can make
money
. . . . . . . .
because your
success is . . . completely automated !
let me show
you how !
click
here
to opt out click here % random _ text



## 初始化变量：从最原始文档，到读取内容，到训练集labels

In [4]:
import glob
import os
emails, labels =[], []
file_path = 'enron1/spam/'
for filename in glob.glob(os.path.join(file_path, '*.txt')):     # 遍历垃圾邮件的文档list
    with open(filename, 'r', encoding = 'ISO-8859-1') as infile:
        emails.append(infile.read())                             # 读取邮件内容，并存入email list
        labels.append(1)                                         # 垃圾邮件的label 指定为1，合法邮件为0

# ham email:
ham_file_path = 'enron1/ham/'
for filename in glob.glob(os.path.join(ham_file_path, '*.txt')):
    with open(filename, 'r', encoding = 'ISO-8859-1') as infile:
        emails.append(infile.read())
        labels.append(0)

## 清洗原始文本数据：

    * 清除数字和标点符号
    * 移除人名（可选）
    * 移除停用词
    * 词形还原

In [5]:
from nltk.corpus import names
from nltk.stem import WordNetLemmatizer

def letters_only(astr):
    return astr.isalpha()

all_names = set(names.words())
lemmatizer = WordNetLemmatizer()

# 将上述method 合为一个function 来做文本清洗：

def clean_text(docs): # 参数输入为emails list
    cleaned_docs = []
    for doc in docs:
        cleaned_docs.append(' '.join([lemmatizer.lemmatize(word.lower())
                                     for word in doc.split()
                                     if letters_only(word) and word not in all_names]))
    return cleaned_docs

cleaned_emails = clean_text(emails)

移除停用词，提取特征，**向量化**：

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english', max_features=500) # 只考虑最频繁的500 terms；可调整获得更好的accuracy
term_docs = cv.fit_transform(cleaned_emails) # 向量化，将document matrix（rows of words）转换为一个 term document matrix
print(term_docs[0])                          # 每行是一个文档、一份email的词频稀疏向量

  (0, 481)	1
  (0, 357)	1
  (0, 69)	1
  (0, 285)	1
  (0, 424)	1
  (0, 250)	1
  (0, 345)	1
  (0, 445)	1
  (0, 231)	1
  (0, 497)	1
  (0, 47)	1
  (0, 178)	2
  (0, 125)	2



    (row index, feature/term index) 词频      ——该稀疏向量的形式
查看对应的term：

In [7]:
feature_names = cv.get_feature_names()
feature_names[481]  # 上面第一个term
feature_names[357]

'website'

'read'

In [8]:
# 或者查看vocabulary dictionary：{feature: feature_index}
feature_mapping=cv.vocabulary_
#feature_mapping
feature_mapping['read']

357

## 得到term_docs后，就可以建模了，用Naive Bayes model

分别计算prior, likelihood, evidence

![naive bayes](nb_model.png)

### 将数据按label分组：

In [9]:
def get_label_index(labels):
    from collections import defaultdict # defaultdict: 果希望dict 的key不存在时，返回一个默认值
    label_index = defaultdict(list)     # 相当于 label_index ={} 空字典，只不过下面引用key时，若key不存在，则返回值为列表
    for index, label in enumerate(labels):
        label_index[label].append(index)  # test example见下文
    return label_index

In [10]:
d = dict(a=[1,2,3])
d
d['a'].append(4)
d

{'a': [1, 2, 3]}

{'a': [1, 2, 3, 4]}

In [11]:
label_index = get_label_index(labels) # 其实就是ham/spam : 0/1 2组
# 训练样本的indices 按0/1类分组 {0: [3000, 3001, 3002, 3003, ...... 6670, 6671], 1: [0, 1, 2, 3, ...., 2998, 2999]}

### 计算prior：

In [12]:
def get_prior(label_index):
    """Compute prior based on training samples
    Args:
        label_index (grouped sample indices by class)
    Return:
        dictionary, with class label as key, corresponding prior as the value
    """
    prior = {label: len(index_list) for label, index_list in label_index.items()} # {"0":3672,"1":1500}
    total_count = sum(prior.values())  # sum(dict_values([3672,1500]))
    for label in prior:
        prior[label] /= float(total_count)
    return prior

In [13]:
prior = get_prior(label_index)
prior

{0: 0.7099767981438515, 1: 0.2900232018561485}

### 计算likelihood：

In [14]:
import numpy as np

def get_likelihood(term_document_matrix, label_index, smoothing=0):
    """Compute likelihood based on training samples.
    Args:
        term_document_matrix (sparse matrix)
        label_index (grouped sample indices by class)
        smoothing (integer additive Laplace smoothing parameter)
    Returns:
        dictionary, with class as key, corresponding conditional probability P(feature|class) vector as value
    """
    likelihood = {}
    for label, index in label_index.items():
        likelihood[label] = term_document_matrix[index, :].sum(axis=0) + smoothing
        likelihood[label] = np.asarray(likelihood[label])[0]
        total_count = likelihood[label].sum()
        likelihood[label] = likelihood[label] / float(total_count)
    return likelihood

In [15]:
likelihood = get_likelihood(term_docs, label_index, smoothing=1)
len(likelihood[0])

500

### 有了prior 和likelihood，就能计算test／new sample的后验概率：
一个trick：避免连乘导致下溢，取log 变成连加再对结果取e值

In [23]:
def get_posterior(term_document_matrix, prior, likelihood):
    """Computer posterior of testing samples, based on prior and likelihood
    Args:
        term_document_matrix (sparse matrix)
        prior (dictionary, with class label as key, corresponding prior as the value)
        likelihood (dictionary, with class label as key, corresponding conditional probability vector as value)
    Return:
        dictionary, with class label as key, corresponding posterior as value
    """
    num_docs = term_document_matrix.shape[0]
    posteriors = []
    for i in range(num_docs):
        # posterior is proportional to prior * likelihood
        # = exp(log(prior * likelihood))
        # = exp(log(prior) + log(likelihood))
        posterior = {key: np.log(prior_label) for key, prior_label in prior.items()}
        for label, likelihood_label in likelihood.items():
            term_document_vector = term_document_matrix.getrow(i)
            counts = term_document_vector.data
            indices = term_document_vector.indices
            for count, index in zip(counts, indices):
                posterior[label] += np.log(likelihood_label[index]) * count
        # exp(-1000)/exp(-999) 会导致除零错误，但它= exp(0)/exp(1) = exp(-1)
        min_log_posterior = min(posterior.values())
        for label in posterior:
            try:
                posterior[label] = np.exp(posterior[label] - min_log_posterior)
            except:
                # 如果一个log值极大，就设其为无穷大
                posterior[label] = float('inf')
        # normalize so that all sum up to 1
        sum_posterior = sum(posterior.values())
        for label in posterior:
            if posterior[label] == float('inf'):
                posterior[label] = 1.0
            else:
                posterior[label] /= sum_posterior
        posteriors.append(posterior.copy())
    return posteriors

### 预测new sample:

In [29]:
emails_test = [
    '''Subject: flat screens
    hello ,
    please call or contact regarding the other flat screens requested .
    trisha tlapek - eb 3132 b
    michael sergeev - eb 3132 a
    also the sun blocker that was taken away from eb 3131 a .
    trisha should two monitors also michael .
    thanks
    kevin moore''',
    '''Subject: having problems in bed ? we can help !
    cialis allows men to enjoy a fully normal sex life without having to plan the sexual act .
    if we let things terrify us , life will not be worth living .
    brevity is the soul of lingerie .
    suspicion always haunts the guilty mind .''',
]

cleaned_test = clean_text(emails_test)
term_docs_test = cv.transform(cleaned_test)
posterior = get_posterior(term_docs_test, prior, likelihood)
print(posterior)

[{1: 0.0045311245507072767, 0: 0.99546887544929274}, {1: 0.99963843948151876, 0: 0.00036156051848121361}]


第一个邮件是合法邮件，第二个邮件是垃圾邮件

### 模型评估
拆分数据集

In [30]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(cleaned_emails, labels, test_size=0.33, random_state=42)

len(X_train),len(Y_train)
len(X_test), len(Y_test)

(3465, 3465)

(1707, 1707)

In [31]:
term_docs_train = cv.fit_transform(X_train)
label_index = get_label_index(Y_train)
prior = get_prior(label_index)
likelihood = get_likelihood(term_docs_train, label_index, smoothing=1)

term_docs_test = cv.transform(X_test)
posterior = get_posterior(term_docs_test, prior, likelihood)

/Users/linzhun/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:28: RuntimeWarning: overflow encountered in exp


评估：

In [27]:
correct = 0.0
for pred, actual in zip(posterior, Y_test):
    if actual == 1:
        if pred[1] >= 0.5:
            correct += 1
    elif pred[0] > 0.5:   # 可省略 if actual ==0
        correct += 1
print('The accuracy on {0} testing samples is: {1:.1f}%'.format(len(Y_test), correct/len(Y_test)*100))

The accuracy on 1707 testing samples is: 92.0%


### 从头代码实现一个模型，是学习机器学习模型的最好方式。当然，我们也可以走捷径，调用sklearn :